In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
from datetime import datetime

# keras functions
from keras import callbacks

# our libraries
from preprocessing import *

from cnn_models import *
from evaluate import *

%load_ext autoreload
%autoreload 2

#### Overview
The goal here is to use the CNN to reduce the size of the input image to obtain a "discretized" image of shape, e.g. (W/16, H/16). Every entry of this image is related to a patch in the input image. This obtained image is compared by the CNN with the groundtruth (after properly discretizing by it patch-wise).

### - Load data

In [ ]:
# Loaded a set of images
n = 2

imgs, gt_imgs = load_images(n)
imgs[0].shape, gt_imgs[0].shape

### - Convert the data to the correct format
We reshape each input to fulfill the requirements of the tensorflow library. 

In [ ]:
# set predict_patch_width in accordance to the model: the model is classifying 
# patch-wise, patches of size predict_patch_width x predict_patch_width
predict_patch_width = 8

X, Y = images_to_XY(imgs, gt_imgs, predict_patch_width=predict_patch_width)
X.shape, Y.shape

### - For now avoid cross validation, just split the datasest in test and train. 

In [ ]:
test_ratio = 0.5

train, test = split_train_test(X, Y, test_ratio=test_ratio, seed=1)
train.X.shape, train.Y.shape, test.X.shape, test.Y.shape 

In [ ]:
# # check it makes sense (show the i-th input of set_)
# i = 0
# set_ = test

# fig, axs = plt.subplots(1, 2, figsize=(20, 10))
# axs[0].imshow(set_.Y[i, :, :, 1], cmap='gray')
# axs[1].imshow(set_.X[i, :, :])

### - Build the CNN model
Choose one of the models you defined and the compile it.

In [ ]:
model = model1()

In [ ]:
model = model2()

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### - Fit the model on the train data

In [ ]:
# some callbacks example: 

# create a list of callbacks we want to use during training
# # a callback to store epoch results to a csv file
# filename='model_train_new.csv'
# csv_log = callbacks.CSVLogger(filename, separator=',', append=False)

# # a callback to stob before doing the predefined number of epochs (stop before overfitting the data)
# early_stopping = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='min')

# # a callback to save the best model (best model = the one with the lowest 'monitor' variable)
# filepath = "best-weights-{epoch:03d}-{loss:.4f}-{acc:.4f}.hdf5"
# checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

# # callbacks_list = [csv_log,early_stopping,checkpoint]

In [ ]:
# use just the callback to save the best model (best model = the one with the best 'monitor' variable)
filepath = "models/"+str('{0:%Y-%m-%d_%H:%M:%S}'.format(datetime.now()))+"_best-weights.hdf5" # -{epoch:03d}-{loss:.4f}-{acc:.4f}
checkpoint = callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

# either validation_data=(test.X, test.Y) or validation_split=0.2
num_epoch = 10
hist = model.fit(
    train.X, train.Y, 
    batch_size = 32, 
    epochs = num_epoch, 
    verbose = 1, 
    validation_data = (test.X, test.Y), 
    callbacks = callbacks_list
)

#### Plot the accuracy and the loss obtained during training

In [ ]:
plot_history(hist)

#### Display the output of a specific layer

In [ ]:
# these are all the layers 
model.layers

In [ ]:
# choose a layer and an image 
image = test.X[0]
layer_num = 8

show_layer_output(model, image, layer_num, filename="") # pass a filename if you want to store the image to file 

### - Evaluate the model on the test data

In [ ]:
# check the performance on train or test
set_ = train

evaluate_model(model, set_.X, set_.Y)

### - Show a prediction

In [ ]:
# choose an image to predict (or part of it)
img = train.X[0][:, :]

display_prediction(model, img, ax=None)

### - Save/load model

In [ ]:
# # Saving and loading model and weights
# from keras.models import model_from_json
# from keras.models import load_model

# # serialize model to JSON
# model_json = model.to_json()
# with open("model.json", "w") as json_file:
#     json_file.write(model_json)
# # serialize weights to HDF5
# model.save_weights("model.h5")
# print("Saved model to disk")

# # load json and create model
# json_file = open('model.json', 'r')
# loaded_model_json = json_file.read()
# json_file.close()
# loaded_model = model_from_json(loaded_model_json)
# # load weights into new model
# loaded_model.load_weights("model.h5")
# print("Loaded model from disk")

# model.save('model.hdf5')
# loaded_model=load_model('model.hdf5')

### - Others

In [ ]:
print(
    "-get configurations:", "\n",
    model.get_config(), "\n",
    model.layers[0].get_config(), "\n",

    "\n-get shapes", "\n",
    model.layers[0].input_shape, "\n",
    model.layers[0].output_shape, "\n",
    
    "\n-get weights", "\n",
    model.layers[0].get_weights()[0].shape, "\n",
    
    "\n-check if trainable", "\n",
    model.layers[0].trainable, "\n", # you can set this to false to "freeze" a layer
)